This notebook is part of *The 2019 Australian Open Radar Science Course*.

Copyright (c) Kai Muehlbauer.

Distributed under the BSD 2-Clause "Simplified" License. See LICENSE for more info.

# A Taste of wradlib

As you already know, $\omega radlib$ is a rather low level library. In this notebook some of the features are shown, which should give you an first impression on how to use $\omega radlib$. 

For a more in-depth look into the capabilities of wradlib, please use the notebooks provided in the [VM](https://openradarscience.org/vm-docs/). You also might just start at [wradlib.org](https://wradlib.org/).

## Install TQDM inside current Jupyter kernel

Only for use inside the ORVM  where tqdm is not available. Uncomment the two lines in the next cell and run it. This will install `tqdm` in your current Jupyter kernel. 

In [ ]:
#import sys
#!conda install --yes --prefix {sys.prefix} tqdm

## Import needed python packages

In [ ]:
import wradlib as wrl
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as pl
import numpy as np
import xarray as xr
import glob
import os
try:
    from tqdm import tqdm_notebook as tqdm
except ImportError:
    def tqdm(val, **kwargs):
        print("wradlib: Please wait for completion of time consuming task! \n"
              "wradlib: Please install 'tqdm' for showing a progress bar "
              "instead.")
        return val
#from tqdm import tqdm_notebook as tqdm
try:
    get_ipython().magic("matplotlib inline")
except:
    pl.ion()

## Get hold of some  some Australien Radar Data

The `filepath` and the `filename` below have to be set to the correct folder and the correct naming scheme. It is assumed, that data from IDR71 (Sidney) from 20th of December 2018 is used in this notebook.

In [ ]:
filepath = os.path.expanduser('~/data/aus/terryhills')
print(filepath)
filename = '71_*.pvol.h5'
idr71 = glob.glob(os.path.join(filepath, filename))
idr71.sort()
print("Files available: {}".format(len(idr71)))

# Claim one volume file

This reads one ODIM_H5 volume file into an xarray backed data structure. Place the cursor inside the `OdimH5` parantheses and press `SHIFT-TAB` to inspect the function parameters. Or just have a look [here](https://docs.wradlib.org/en/stable/generated/wradlib.io.xarray.OdimH5.html).

`dim0=azimuth` means that the first dimension of the dataset will be `azimuth` instead of the CfRadial standard `time`. This has some advantages with ODIM_H5 because the data is azimuth-aligned (0-360 deg in most cases).

Set the index (now 11, counting from the whole day of 240 files) to a reasonable value.

You can try to use keyword `chunks={}`, to read the data into dask chunks if `dask` is available. Especially interesting for the TimeSeries example.

In [ ]:
odh = wrl.io.OdimH5(idr71[11], dim0='azimuth')#, chunks={})

## Inspect the data

### CfRadial-like root-object

The `odh` object contains an overview of the contained data in the `root` variable. This is strongly connected to CfRadial standard.

In [ ]:
odh.root

## Looking at the sweeps

### Sweep groups

In [ ]:
list(odh)

### Sweep angles

In [ ]:
odh.root.sweep_fixed_angle

### Inspect one sweep

All relevant and needed dimensions and coordinates as well as the radar moments itself are combined into one xarray Dataset. You can also play with the other sweeps by subsetting `odh` accordingly.

All Xarray features (selecting, indexing, ufuncs etc) can be used with the sweep datasets. 

In [ ]:
swp1 = odh['sweep_1'] 
swp1

### Inspect one moment

Same here, all Xarray features can be used with the DataArray.

In [ ]:
swp1.DBZH

### Simple Plots using Xarray machinery

In [ ]:
odh['sweep_1'].DBZH.plot()

In [ ]:
odh['sweep_1'].DBZH.sortby('time').plot(y='time')

## Georeferencing sweeps

`xyz`-Coordinates of all radar bins are added to the Dataset in Azimthal Equidistant Projection with the radar as center. Also `gr` (ground range), `rays` and `bins` are added for nicer plotting.

In [ ]:
swp1 = swp1.pipe(wrl.georef.georeference_dataset)
swp1

### Simple georeferenced plot

In [ ]:
fig = pl.figure(figsize=(10,8))
swp1.DBZH.plot.pcolormesh(x='x', y='y', cmap='viridis', vmin=0)
pl.gca().set_aspect('equal')

### Use wradlib-Accessor to create curvelinear grid plot

In [ ]:
fig = pl.figure(figsize=(10,8))
qm = swp1.DBZH.wradlib.plot_ppi(proj='cg', fig=fig, vmin=0)
pl.colorbar(qm)

### Use cartopy machinery to plot on map projections

In [ ]:
lat = swp1.latitude.values
lon = swp1.longitude.values

In [ ]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

#### Just AEQD, no fancy stuff

In [ ]:
map_proj = ccrs.AzimuthalEquidistant(central_latitude=lat, central_longitude=lon)
pm = swp1.DBZH.wradlib.plot_ppi(proj=map_proj)
ax = pl.gca()
ax.gridlines()
print(ax)

#### Mercator

In [ ]:
map_proj = ccrs.Mercator(central_longitude=lon)
fig = pl.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=map_proj)
pm = swp1.DBZH.wradlib.plot_ppi(ax=ax)
ax.gridlines(draw_labels=True)

#### Mercator using cartopy features

In [ ]:
import cartopy.feature as cfeature
def plot_lines(ax):
    coast = cfeature.NaturalEarthFeature(category='physical',
                                           name='coastline',
                                           scale='10m',
                                           facecolor='none')
    ax.add_feature(coast, edgecolor='black', lw=2, zorder=4)

map_proj = ccrs.Mercator(central_longitude=lon)
fig = pl.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=map_proj)

pm = swp1.DBZH.wradlib.plot_ppi(ax=ax)
plot_lines(ax)
ax.gridlines(draw_labels=True)

## Quasi Vertical Profile (QVP)

This example shows how to create a so called QVP. We need to define a function to add a height coordinate for plotting.

In [ ]:
print(xr.__version__)

In [ ]:
def add_height(ds):
    ds = ds.pipe(wrl.georef.georeference_dataset)
    height = ds.z.mean('azimuth')
    #ds = ds.assign_coords({'height': (['range'], height)})
    ds = ds.assign_coords(height=(['range'], height))
    return ds

### Single Profile

Here we add the height coordinate and calculate the `mean` over the azimuth using the sweep with the highest available elevation.

In [ ]:
swp14 = odh['sweep_14'].pipe(add_height)
qvp = swp14.mean('azimuth')
qvp

In [ ]:
qvp.DBZH.plot(y='height')

### TimeSeries QVP

The following is preliminary code which is currently implemented in wradlib and will be available in the next version. There are some glitches which have to be fixed, yet, see below.

This loads multiple volumes and combines the sweeps (here only one sweep) along a new time dimension.

In [ ]:
ts = {}
ts['fh'] = []
ts['swp'] = []
for f in tqdm(idr71, desc='Collecting', unit=' Files'):
    fh = wrl.io.OdimH5(f, dim0='azimuth', chunks=None)
    ts['fh'].append(fh)
    ds = fh['sweep_14']
    # reassign time 
    ds = ds.rename({'time': 'rtime'})
    #ds = ds.assign_coords({'time': (['time'], [ds['rtime'].min().values])})
    ds = ds.assign_coords(time= (['time'], [ds['rtime'].min().values]))
    ts['swp'].append(ds)

#### Concat Datasets along time

In [ ]:
ts['swp'] = xr.concat(ts['swp'], 'time')

#### Fix some coordinates

In [ ]:
# fix latitude, longitude
#ts['swp'] = ts['swp'].assign_coords({'longitude': ts['swp'].longitude.min(),
#                                     'latitude': ts['swp'].latitude.min()})
ts['swp'] = ts['swp'].assign_coords(longitude=ts['swp'].longitude.min(),
                                    latitude=ts['swp'].latitude.min())

#### Georeference and add height coordinate

In [ ]:
ts['swp'] = ts['swp'].pipe(wrl.georef.georeference_dataset).pipe(add_height)
ts['swp']

#### Calculate QVP

In [ ]:
ts_qvp = ts['swp'].mean('azimuth')
ts_qvp

#### Plot QVP using discrete colorbar

In [ ]:
fig = pl.figure(figsize=(12,8))
levels = np.arange(-30, 80, 5)
ts_qvp.DBZH.plot(x='time', y='height', cmap='viridis', levels=levels)
pl.gca().set_title('Quasi Vertical Profile')

## Simple Clutter Filter

This extracts clutter information using a approach published by Gabella.

### Define function to be used with Xarray Dataset

In [ ]:
def extract_clutter(ds):
    clmap = wrl.clutter.filter_gabella(ds.DBZH.values,
                                       wsize=5,
                                       thrsnorain=0.,
                                       tr1=6.,
                                       n_p=8,
                                       tr2=1.3, 
                                       rm_nans=False)
    ds = ds.assign({'CMAP': (ds.DBZH.dims, clmap)})
    return ds

### Pipe the function, which returns a Dataset with added `CMAP` 

In [ ]:
swp1 = swp1.pipe(extract_clutter)
swp1

In [ ]:
fig = pl.figure(figsize=(12,8))
swp1.DBZH.wradlib.plot(ax=121, fig=fig)
swp1.CMAP.wradlib.plot(ax=122, fig=fig)

## Dual-Pol retrievals

This is just one example for using dual pol moments. Here we facilitate `RHOHV` and `ZDR` to calculate Depolarization Ratio.

### Depolarizaton Ratio

We create the function, to be used with the Xarray Dataset, pipe it and create the diagnostic plot.

In [ ]:
def depol(ds):
    dep = wrl.dp.depolarization(ds.ZDR.values,
                                ds.RHOHV.values)
    ds = ds.assign({'DR': (ds.DBZH.dims, dep)})
    return ds

In [ ]:
swp1 = swp1.pipe(depol)
swp1

In [ ]:
fig = pl.figure(figsize=(12,8))
qm = swp1.ZDR.wradlib.plot(ax=131, fig=fig)
pl.colorbar(qm, pad=0.05, shrink=0.35)
qm = swp1.RHOHV.wradlib.plot(ax=132, fig=fig)
pl.colorbar(qm, pad=0.05, shrink=0.35)
qm = swp1.DR.wradlib.plot(ax=133, fig=fig)
pl.colorbar(qm, pad=0.05, shrink=0.35)

## Create 3D-Volume

This is just one example of creating a 3D representation of the volume data.

### Iterate over the sweeps and extract coordinates and data

This takes the radar location and the given projection into account. Here: using UTM65S, for Sydney radar. Change accordingly for other radar locations.

In [ ]:
from osgeo import osr
proj = osr.SpatialReference()
proj.ImportFromEPSG(32756)
xyz, data = np.array([]).reshape((-1, 3)), np.array([])
for swp in odh.values():
    xyz_ = wrl.vpr.volcoords_from_polar(odh.location, swp.fixed_angle.values,
                                        swp.azimuth.values, swp.range.values, proj=proj)
    xyz, data = np.vstack((xyz, xyz_)), np.append(data, swp.DBZH.values.ravel())

### Initialize Volume Parameters

Try with the given set of parameters. If the processing time is small, then increase `maxalt` until you see the storm top. Increase horizontal and vertical resolution for more details.

In [ ]:
import datetime as dt
# generate 3-D Cartesian target grid coordinates
maxrange = 200000.
minelev = 0.1
maxelev = 25.
maxalt = 5000.
horiz_res = 2000.
vert_res = 250.
trgxyz, trgshape = wrl.vpr.make_3d_grid(odh.location, proj, maxrange,
                                        maxalt, horiz_res, vert_res)

# interpolate to Cartesian 3-D volume grid
tstart = dt.datetime.now()
gridder = wrl.vpr.CAPPI(xyz, trgxyz, trgshape, maxrange, minelev,
                        maxelev)
vol = np.ma.masked_invalid(gridder(data).reshape(trgshape))
print("3-D interpolation took:", dt.datetime.now() - tstart)

In [ ]:
# diagnostic plot
trgx = trgxyz[:, 0].reshape(trgshape)[0, 0, :]
trgy = trgxyz[:, 1].reshape(trgshape)[0, :, 0]
trgz = trgxyz[:, 2].reshape(trgshape)[:, 0, 0]
wrl.vis.plot_max_plan_and_vert(trgx, trgy, trgz, vol, unit="dBZH",
                               levels=range(0, 100))